In [3]:
import sys
import os
import numpy as np
import pandas as pd
from FCLayer import FcLayer
from ACLayer import AcLayer
from Network import Network
from activation_functions import sigmoid, relu, sigmoid_prime, tanh, tanh_prime,relu_prime
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [4]:
# Set random seed for reproducibility
np.random.seed(42)

df_vote = pd.read_csv("CongressionalVotingID.shuf.lrn.csv", index_col="ID")
df_vote = df_vote.applymap(lambda x: 1 if x == "y" else 0 if x == "n" else x)
df_vote = df_vote.replace("democrat", 1)
df_vote = df_vote.replace("republican", 0)
df_vote = df_vote.replace("unknown", 0)



In [5]:
x_train, x_test, y_train, y_test = train_test_split(df_vote.iloc[:,1:], df_vote.iloc[:, 0],test_size=0.4, random_state=11)

In [6]:
x_train = np.expand_dims(x_train.to_numpy(), axis=1)
y_train = np.expand_dims(y_train.to_numpy(), axis=1)
x_test = np.expand_dims(x_test.to_numpy(), axis=1)
y_test = np.expand_dims(y_test.to_numpy(), axis=1)

In [8]:
np.unique(y_test)
y_train.shape

(130, 1)

In [ ]:

learning_rate = 0.1


network1 = Network(learning_rate)
input_values = 16
output_values = 1
intermediate_values = 3
layers = 3
epochs = 1000

for j in range(layers):
    i = intermediate_values
    o = intermediate_values

    if not j:
        i = input_values
    elif j + 1 == layers:
        o = output_values

    fc = FcLayer(i, o)
    ac = AcLayer(tanh, tanh_prime)
    network1.add(fc)
    network1.add(ac)

network1.train(x_train, y_train, epochs=epochs)

out = network1.predict(x_train)

In [ ]:
y_pred=network1.predict(x_test)
y_pred=np.where(np.array(y_pred) > 0.5, 1, 0)
np.unique(y_pred)

***Classification report for Self Neural Network***

In [ ]:
print(classification_report(y_pred=np.squeeze(y_pred),y_true=np.squeeze(y_test)))

***Tensor Flow***

In [281]:
def train_model(x_train,y_train, num_nodes, dropout_prob,lr, batch_size, epochs):
    nn_model=tf.keras.Sequential([tf.keras.layers.Dense(num_nodes, activation='relu', input_shape=(16,)),
                              tf.keras.layers.Dropout(dropout_prob),
                             tf.keras.layers.Dense(num_nodes, activation='relu'),
                             tf.keras.layers.Dropout(dropout_prob),
                             tf.keras.layers.Dense(1, activation='sigmoid')])

    nn_model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss="binary_crossentropy",metrics=['accuracy'])

    #x_train=x_train.astype(float)

    history = nn_model.fit(x_train,y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
    return nn_model, history

In [282]:
nn_model, history= train_model(np.squeeze(x_train),np.squeeze(y_train),16 , 0,0.1, 32, 1000)

In [283]:
y_pred=nn_model.predict(np.squeeze(x_test))
y_pred=(y_pred>0.5).astype(int).reshape(-1)
y_pred


print(classification_report(y_pred,np.squeeze(y_test)))

3/3 [==============================] - 0s 914us/step
              precision    recall  f1-score   support

           0       0.97      0.95      0.96        38
           1       0.96      0.98      0.97        50

    accuracy                           0.97        88
   macro avg       0.97      0.96      0.97        88
weighted avg       0.97      0.97      0.97        88

